In [1]:
from copy import deepcopy

import progressbar
import spacy
import os
w2v_model = spacy.load('en_core_web_lg')

In [2]:
from dotenv import load_dotenv
from pathlib import Path

initial_wd = os.getcwd()
while os.path.split(os.getcwd())[-1] != 'scientific-knowledge-distiller':
    os.chdir(os.path.join(os.getcwd(), '..'))
root_path = os.getcwd()
os.chdir(initial_wd)

load_dotenv(dotenv_path=Path(os.path.join(root_path, '.env')))

True

In [3]:
query = 'out-of-distribution detection in deep neural networks'
limit = 100
threshold_sim = 0.0

In [4]:
from search_engine.databases.database_client import SupportedSources
from search_engine import Search

s = Search(
    query,
    limit=limit,
    sources=(
        SupportedSources.ARXIV,
        SupportedSources.CROSSREF,
        SupportedSources.INTERNET_ARCHIVE,
        SupportedSources.SEMANTIC_SCHOLAR,
        SupportedSources.UNPAYWALL,
    )
)
s.perform()

INFO:arxiv.arxiv:Requesting 100 results at offset 0
INFO:arxiv.arxiv:Requesting page of results
INFO:root:unpaywall: 4
INFO:arxiv.arxiv:Got first page; 2071017 of inf results available
INFO:root:arXiv: 1
INFO:root:arXiv: 2
INFO:root:arXiv: 3
INFO:root:arXiv: 4
INFO:root:arXiv: 5
INFO:root:arXiv: 6
INFO:root:arXiv: 7
INFO:root:arXiv: 8
INFO:root:arXiv: 9
INFO:root:arXiv: 10
INFO:root:arXiv: 11
INFO:root:arXiv: 12
INFO:root:arXiv: 13
INFO:root:arXiv: 14
INFO:root:arXiv: 15
INFO:root:arXiv: 16
INFO:root:arXiv: 17
INFO:root:arXiv: 18
INFO:root:arXiv: 19
INFO:root:arXiv: 20
INFO:root:arXiv: 21
INFO:root:arXiv: 22
INFO:root:arXiv: 23
INFO:root:arXiv: 24
INFO:root:arXiv: 25
INFO:root:arXiv: 26
INFO:root:arXiv: 27
INFO:root:arXiv: 28
INFO:root:arXiv: 29
INFO:root:arXiv: 30
INFO:root:arXiv: 31
INFO:root:arXiv: 32
INFO:root:arXiv: 33
INFO:root:arXiv: 34
INFO:root:arXiv: 35
INFO:root:arXiv: 36
INFO:root:arXiv: 37
INFO:root:arXiv: 38
INFO:root:arXiv: 39
INFO:root:arXiv: 40
INFO:root:arXiv: 41
INFO

In [5]:
results = list(s.results())

total found: 404
starting deduplication...


In [7]:
final_results = []
embedding1 = w2v_model(query)

for doc in progressbar.progressbar(results):
    if not doc.title:
        continue

    title = doc.title.lower()
    embedding2 = w2v_model(title)
    sim_score = embedding1.similarity(embedding2)

    if sim_score > threshold_sim:
        final_results.append((deepcopy(doc), sim_score))
final_results = sorted(final_results, key=lambda x: -x[1])

100% (385 of 385) |######################| Elapsed Time: 0:00:01 Time:  0:00:01


In [10]:
import pandas as pd

df = pd.DataFrame(final_results, columns=['Title', 'Sim'])

In [11]:
df

,Title,Sim
0,Uncertainty-Based Out-of-Distribution Detectio...,0.973442
1,Linking Neural Collapse and L2 Normalization w...,0.968012
2,Efficient Out-of-Distribution Detection in Dig...,0.961183
3,Out-of-Distribution Example Detection in Deep ...,0.948399
4,Ramifications of Approximate Posterior Inferen...,0.947772
...,...,...
374,Do Deep Neural Networks Contribute to Multivar...,0.567068
375,Deep Learning,0.550949
376,Have You Stolen My Model? Evasion Attacks Agai...,0.543352
377,Detecting Gas Vapor Leaks Using Uncalibrated S...,0.527859
